In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [58]:
train = pd.read_csv('train_v2.csv')
train.head()

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev,potential,pack_dens,volume,group15,group17,int_atm_dist
0,1.0,33.0,80.0,0.6250,0.3750,0.000,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.0680,3.4387,-31.454381,6.201844,621.360462,32.0,48.0,6.146797
1,2.0,194.0,80.0,0.6250,0.3750,0.000,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.2490,2.9210,-52.179568,8.993174,3044.057423,32.0,48.0,8.935653
2,3.0,227.0,40.0,0.8125,0.1875,0.000,9.7510,5.6595,13.9630,90.9688,91.1228,30.5185,0.1821,2.7438,-53.636758,6.580119,1015.527437,16.0,24.0,6.458938
3,4.0,167.0,30.0,0.7500,0.0000,0.250,5.0036,5.0034,13.5318,89.9888,90.0119,120.0017,0.2172,3.3492,-50.088740,5.664797,639.459800,12.0,18.0,5.515954
4,5.0,194.0,80.0,0.0000,0.6250,0.375,6.6614,6.6612,24.5813,89.9960,90.0006,119.9893,0.0505,1.3793,-49.030109,9.415613,3425.724109,32.0,48.0,9.355149


In [59]:
test = pd.read_csv('test_v2.csv')
test.head()

,id,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,potential,pack_dens,volume,group15,group17,int_atm_dist
0,1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,90.0006,-29.947552,6.539984,728.750084,32,48,6.481714
1,2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,90.0015,-31.620734,6.166898,610.894869,32,48,6.112181
2,3,167,30.0,0.6667,0.1667,0.1667,4.9811,4.9808,13.4799,89.9900,90.0109,120.0014,-50.717980,5.647889,632.683731,12,18,5.499618
3,4,12,80.0,0.5625,0.4375,0.0000,24.3370,6.0091,5.7620,89.9995,103.8581,90.0002,-86.814480,9.143618,3303.118777,32,48,9.029838
4,5,12,80.0,0.1875,0.5000,0.3125,24.6443,6.2906,6.1589,90.0000,104.5929,90.0001,-82.581758,9.324523,3403.103832,32,48,9.208523


In [60]:
print(train.shape)
print(test.shape)

(2400, 20)
(600, 18)


In [61]:
train.id = train.id.astype(int)

In [62]:
train.set_index('id', inplace=True)
test.set_index('id', inplace=True)

In [63]:
train.head(2)

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev,potential,pack_dens,volume,group15,group17,int_atm_dist
id,,,,,,,,,,,,,,,,,,,
1,33.0,80.0,0.625,0.375,0.0,9.9523,8.5513,9.1775,90.0026,90.0023,90.0017,0.068,3.4387,-31.454381,6.201844,621.360462,32.0,48.0,6.146797
2,194.0,80.0,0.625,0.375,0.0,6.1840,6.1838,23.6287,90.0186,89.9980,120.0025,0.249,2.9210,-52.179568,8.993174,3044.057423,32.0,48.0,8.935653


In [64]:
test.head(2)

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,potential,pack_dens,volume,group15,group17,int_atm_dist
id,,,,,,,,,,,,,,,,,
1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,90.0006,-29.947552,6.539984,728.750084,32,48,6.481714
2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,90.0015,-31.620734,6.166898,610.894869,32,48,6.112181


In [65]:
scalar = StandardScaler()

In [66]:
full_train = train.columns.tolist()
full_test = test.columns.tolist()
full_train.remove('formation_energy_ev_natom')
full_train.remove('bandgap_energy_ev')
X = full_train
y = ['formation_energy_ev_natom','bandgap_energy_ev']

In [67]:
X_data = train[X]
y_data = train[y]

In [68]:
X_data[X] = scalar.fit_transform(X_data)

/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:2356: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/pandas/core/frame.py:2326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the doc

In [69]:
X_data.head(2)

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,potential,pack_dens,volume,group15,group17,int_atm_dist
id,,,,,,,,,,,,,,,,,
1,-1.281235,0.810885,0.902265,0.284159,-1.162707,-0.013767,0.774800,-0.626795,-0.180793,-0.452319,-0.185047,0.260726,-0.558672,-0.643835,0.810885,0.810885,-0.533471
2,0.619639,0.810885,0.902265,0.284159,-1.162707,-0.681399,-0.477888,2.024991,-0.168794,-0.453130,0.974936,-0.440920,1.425380,1.456527,0.810885,0.810885,1.433928


In [70]:
test[X] = scalar.fit_transform(test)

In [71]:
test.head(2)

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,potential,pack_dens,volume,group15,group17,int_atm_dist
id,,,,,,,,,,,,,,,,,
1,-1.245661,0.795712,-0.672619,0.633833,0.101962,0.077071,0.984280,-0.516047,-0.160087,-0.459878,-0.263534,0.291157,-0.304732,-0.540337,0.795712,0.795712,-0.282582
2,-1.245661,0.795712,1.389169,-0.257915,-1.146007,-0.035758,0.723122,-0.609149,-0.156070,-0.459508,-0.263497,0.235078,-0.568059,-0.642435,0.795712,0.795712,-0.540963


In [72]:
X1_train, X1_val, y1_train, y1_val = train_test_split(X_data, y_data['formation_energy_ev_natom'], test_size=0.2, shuffle=True)

In [73]:
X2_train, X2_val, y2_train, y2_val = train_test_split(X_data, y_data['bandgap_energy_ev'], test_size=0.2, shuffle=True)

In [74]:
rmsle_score = make_scorer(mean_squared_log_error, greater_is_better=False)

In [75]:
mae_score = make_scorer(mean_absolute_error, greater_is_better=False)

In [76]:
mlp_grid = {
        'hidden_layer_sizes': [20,40,60,80,100,120,150],
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'alpha': [0.0001,0.001,0.01,0.1,1,10,100,1000,10000],
        'batch_size':['auto'],
        'learning_rate':['constant', 'invscaling', 'adaptive'],
        'learning_rate_init':[0.001],
        'power_t': [0.5],
        'alpha': [0.0001],
        'max_iter': [1000,2000],
        'early_stopping': [False],
        'warm_start': [False]       
               }

In [77]:
mlp = MLPRegressor(random_state=0)
mlp_gs1 = GridSearchCV(mlp, param_grid = mlp_grid, n_jobs=-1, cv=3,  scoring = mae_score, verbose=1)

In [78]:
mlp_gs1.fit(X1_train,y1_train)

Fitting 3 folds for each of 504 candidates, totalling 1512 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   38.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:  6.6min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 13.7min
[Parallel(n_jobs=-1)]: Done 1512 out of 1512 | elapsed: 17.0min finished


GridSearchCV(cv=3, error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=0, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [20, 40, 60, 80, 100, 120, 150], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'], 'alpha': [0.0001], 'batch_size': ['auto'], 'learning_rate': ['constant', 'invscaling', 'adaptive'], 'learning_rate_init': [0.001], 'power_t': [0.5], 'max_iter': [1000, 2000], 'early_stopping': [False], 'warm_start': [False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
    

In [83]:
gs1 = [x[1] for x in mlp_gs1.grid_scores_]
gs1_filter = [i for i in gs1 if i > -0.0283]
gs1_filter

/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[-0.028232303333685493,
 -0.028232303333685493,
 -0.028232303333685493,
 -0.028232303333685493,
 -0.028232303333685493,
 -0.028232303333685493,
 -0.028279211332219587,
 -0.028279211332219587,
 -0.028279211332219587,
 -0.028279211332219587,
 -0.028279211332219587,
 -0.028279211332219587]

In [84]:
gs1_model = list()
for x  in mlp_gs1.grid_scores_:
    if x[1] > -0.0283:
        gs1_model.append(x)
gs1_model        

/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/sklearn/model_selection/_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: -0.02823, std: 0.00043, params: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hidden_layer_sizes': 40, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 1000, 'power_t': 0.5, 'solver': 'lbfgs', 'warm_start': False},
 mean: -0.02823, std: 0.00043, params: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hidden_layer_sizes': 40, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 2000, 'power_t': 0.5, 'solver': 'lbfgs', 'warm_start': False},
 mean: -0.02823, std: 0.00043, params: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hidden_layer_sizes': 40, 'learning_rate': 'invscaling', 'learning_rate_init': 0.001, 'max_iter': 1000, 'power_t': 0.5, 'solver': 'lbfgs', 'warm_start': False},
 mean: -0.02823, std: 0.00043, params: {'activation': 'relu', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hidden_layer_siz

In [85]:
mlp = MLPRegressor()
mlp_gs2 = GridSearchCV(mlp, param_grid = mlp_grid, n_jobs=-1, cv=3,  scoring =  mae_score,verbose=1)

In [ ]:
mlp_gs2.fit(X2_train,y2_train)

Fitting 3 folds for each of 504 candidates, totalling 1512 fits


/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/anuj8june/python3env/envs/python3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [44]:
gs2 = [x[1] for x in mlp_gs2.grid_scores_]
gs2_filter = [i for i in gs2 if i > -0.138]
gs2_filter

[-0.13778701667031154,
 -0.13785924246795681,
 -0.13767308452297106,
 -0.13705880660507622,
 -0.13798824019670858,
 -0.13754931893428443,
 -0.13753209377205833,
 -0.1379174772801332,
 -0.13765015984669851,
 -0.13767502832032424,
 -0.13731753363413368,
 -0.13719231392887724]

In [48]:
gs2_model = list()
for x  in mlp_gs2.grid_scores_:
    if x[1] > -0.138:
        gs2_model.append(x)
gs2_model

[mean: -0.13779, std: 0.01179, params: {'activation': 'logistic', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hidden_layer_sizes': 20, 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 2000, 'power_t': 0.5, 'solver': 'lbfgs', 'warm_start': False},
 mean: -0.13786, std: 0.01387, params: {'activation': 'logistic', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hidden_layer_sizes': 20, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 2000, 'power_t': 0.5, 'solver': 'lbfgs', 'warm_start': False},
 mean: -0.13767, std: 0.01250, params: {'activation': 'logistic', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hidden_layer_sizes': 40, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 1000, 'power_t': 0.5, 'solver': 'lbfgs', 'warm_start': False},
 mean: -0.13706, std: 0.01135, params: {'activation': 'logistic', 'alpha': 0.0001, 'batch_size': 'auto', 'early_stopping': False, 'hi

### Model 1 NN

In [66]:
t = test.columns.tolist()
y_test = test.copy()
y_test[t] = scalar.fit_transform(test)
y_test.head(2)

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree
id,,,,,,,,,,,
1,2.293023,5.0,2.5625,3.451693,3.064616,3.025389,4.491563,2.602521,3.090841,2.935899,4.001878
2,2.293023,5.0,4.2500,2.774154,2.000000,2.936887,4.280558,2.525057,3.091556,2.936158,4.001907


In [122]:
m1_nn = MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', early_stopping=False, hidden_layer_sizes=120, learning_rate='constant', learning_rate_init=0.001, max_iter=2000, power_t=0.5, solver='lbfgs', warm_start=False)


In [123]:
m1_nn.fit(X, y['formation_energy_ev_natom'])

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=120, learning_rate='constant',
       learning_rate_init=0.001, max_iter=2000, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [124]:
p1 = m1_nn.predict(y_test)

In [125]:
print(p1.shape)

(600,)


### Model 2 NN

In [126]:
m2_nn = MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', early_stopping=False, hidden_layer_sizes=120, learning_rate='constant', learning_rate_init=0.001, max_iter=500, power_t=0.5, solver='lbfgs', warm_start=False)       

In [127]:
m2_nn.fit(X, y['bandgap_energy_ev'])

MLPRegressor(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=120, learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [128]:
p2 = m2_nn.predict(y_test)

In [129]:
p2.shape

(600,)

In [130]:
test['formation_energy_ev_natom'] = p1
test['bandgap_energy_ev'] = p2

In [131]:
test.head(2)

,spacegroup,number_of_total_atoms,percent_atom_al,percent_atom_ga,percent_atom_in,lattice_vector_1_ang,lattice_vector_2_ang,lattice_vector_3_ang,lattice_angle_alpha_degree,lattice_angle_beta_degree,lattice_angle_gamma_degree,formation_energy_ev_natom,bandgap_energy_ev
id,,,,,,,,,,,,,
1,33,80.0,0.1875,0.4688,0.3438,10.5381,9.0141,9.6361,89.9997,90.0003,90.0006,0.207051,1.668168
2,33,80.0,0.7500,0.2500,0.0000,9.8938,8.5014,9.1298,90.0038,90.0023,90.0015,0.069204,3.997254


In [132]:
dfcsv = test[['formation_energy_ev_natom','bandgap_energy_ev']]
dfcsv.head()

,formation_energy_ev_natom,bandgap_energy_ev
id,,
1,0.207051,1.668168
2,0.069204,3.997254
3,0.147716,3.436736
4,0.035946,3.044159
5,0.143977,1.650324


In [137]:
for i,row in dfcsv[dfcsv['formation_energy_ev_natom'] <= 0.0].iterrows():
    dfcsv.at[i,'formation_energy_ev_natom'] = dfcsv['formation_energy_ev_natom'].mean()

In [139]:
#dfcsv.to_csv('sub2_nn.csv')